Spam Mail-Naive Bayes-CA02
Victoria Capobianco


In [18]:
#import necessary packages
import os #fetching contents from folder
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score



In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#Function 1: Cleaning and Preparing Data

"""This function builds a Dictionary of most common 3000 words from all the email content. 
First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and 
any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by 
keeping only most common 3000 words in the dictionary. It returns the Dictionary."""

def make_Dictionary(root_dir):
#define function name
  all_words = [] 
  #empty list to add all words and symbols in the dictionary
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] 
  #join multiple path components in directory to read folders (f) and return a list of the names of the entries in the directory  
  for mail in emails: #iterate through items in emails list
    with open(mail) as m: #opens mail and returns it as file object, m 
      for line in m: #iterate through each line in each item
        words = line.split() #return list of split strings from each line
        all_words += words #add list of split strings to assigned variable, all_words 
  dictionary = Counter(all_words) #keeps track of how many times each word is added to all_words list
  list_to_remove = list(dictionary) #returns dictionary as list


  for item in list_to_remove: #iterate through each item in list
    if item.isalpha() == False: #identifies if item is non-alpha-numeric and assigns it False
      del dictionary[item] #removes non-alpha-numeric items 
    elif len(item) == 1: #identifies if item is has length 1 (single character alpha numeric)
      del dictionary[item] #removes single character alpha numerics
  dictionary = dictionary.most_common(3000) #shrinks dictionary by keeping most common 3000 items
  return dictionary #returns final dictionary 


In [21]:
#Function 2: Extract Features and Corresponding Label Matrix

"""This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to 
the number of email files). The function also analyzes the File Names of each email file and decides if it's a 
Spame or not based on the naming convention. Based on this the function also creates the Labelled Data Column. 
This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column."""

def extract_features(mail_dir): #define function
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  #join multiple path components in directory to read files (fi) and return a list of names of entries in directory
  features_matrix = np.zeros((len(files),3000)) #returns new array with length of 3000 columns and rows equal to number of email files 
  train_labels = np.zeros(len(files)) #returns new array with rows equal to number of email files and with zeros 
  count = 1; #assign naming convention variable 
  docID = 0; #assign naming convention variable
  for fil in files: #iterate through files
    with open(fil) as fi: #opens file and returns it as file object, f
      for i, line in enumerate(fi): #adds counter as function iterates through files
        if i ==2: #identifies if item is counted 2 times in list
          words = line.split() #splits string into list, words
          for word in words: #iterate through items in list, words
            wordID = 0 #assign naming convention
            for i, d in enumerate(dictionary): #adds counter as function iterates through dictionary
              if d[0] == word: #identifies if item is counted 0 times
                wordID = i #assign naming convention 
                features_matrix[docID,wordID] = words.count(word) #slices array to count words in docID and wordID columns
      train_labels[docID] = 0; #slice array where docID is equal to 0 
      filepathTokens = fil.split('/') #splits strings using / marker
      lastToken = filepathTokens[len(filepathTokens)-1] 
      if lastToken.startswith("spmsg"): #iterate through array to see if it starts with spmsg
        train_labels[docID] = 1; #extracting training dataset 
        count = count + 1 #creating label column
      docID = docID + 1 #creating label column
  return features_matrix, train_labels #return feature dataset and label column


In [22]:
#Training and Test Datasets
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA_02/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA_02/Data/test-mails'

In [23]:
#Main Program 
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naive Bayes algorithm .....")
model.fit(features_matrix, labels)
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))


reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154
